In [ ]:
import random
import numpy as np
import pandas as pd



def generate_storm_chess_stats():
  """Generates a random storm chess stats dataset with the following constraints:

  * The total number of challenges is 10.
  * There are only 3 available hints during each challenge.
  * The maximum time for each challenge is 10 minutes.

  Returns:
    A Pandas DataFrame containing the generated data.
  """

  # Generate random player and agent Elo ratings.
  player_elo = np.random.randint(700, 3000, )
  agent_elo = np.random.randint(700, 3000, )

  # Generate random number of solved storm chess challenges out of 10.
  solved = np.random.randint(0, 10, )

  # Generate random average completion time for each of the solved challenges (in minutes).
  avg_time = np.random.randint(1, 10,)

  # Generate random average number of hints used for each of the solved challenges.
  avg_hints = np.random.randint(0, 3,)

  # Generate random number of skipped challenges out of the 10 challenges.
  skipped = np.random.randint(0, 10,)

  # Calculate the expected score for each player based on their Elo ratings.
  expected_score = 1 / (1 + 10**((agent_elo - player_elo) / 400))

  # Calculate the actual score for each player based on the number of solved challenges.
  actual_score = solved / 10

  # Calculate the new Elo rating for each player.
  K = 32 if player_elo < 2400 else 16
  new_elo = round(player_elo + K * (expected_score - actual_score))

  return [player_elo , agent_elo , solved , avg_hints , avg_time  , skipped , new_elo]




if __name__ == "__main__":
  
  dataset = []
  for sample in range(1000000):
    dataset.append(generate_storm_chess_stats())
  df = pd.DataFrame(dataset,columns=["elo", "agent_elo", "solved", "avg_hints", "avg_time", "skipped", "new_elo"])
  # Save the dataset to a CSV file.
  df.to_csv("C:\\Users\\faisa\\Documents\\Games_Section\\chess\\CSVs\\storm_chess_stats.csv", index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Load your dataset
data = pd.read_csv("C:\\Users\\faisa\\Documents\\Games_Section\\chess\\CSVs\\storm_chess_stats.csv")

# Split the data into features (X) and the target variable (y)
X = data.drop(columns=["new_elo"])
y = data["new_elo"]

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Create a RandomForestRegressor object
model = RandomForestRegressor(n_estimators=150, max_depth=100, n_jobs=-1)

# Fit the RandomForestRegressor object to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

joblib.dump(model, "storm_chess_regressor.pkl")

print(f"Mean squared error: {mse}")
print(f"R-squared score: {r2}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
# Load your dataset
data = pd.read_csv("C:\\Users\\faisa\\Documents\\Games_Section\\chess\\CSVs\\storm_chess_stats.csv")

# Split the data into features (X) and the target variable (y)
X = data.drop(columns=["new_elo"])
y = data["new_elo"]

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

type(X_test.head(1))

# Usage

In [6]:
import joblib
from flask import Flask, request, jsonify
import pandas as pd
import stockfish
import chess

model = joblib.load("storm_chess_regressor.pkl")
stockfish_path = "C:\\Users\\faisa\\Documents\\Games_Section\\chess\\AIs\\stockfish\\stockfish-windows-x86-64-avx2.exe"

app = Flask(__name__)

@app.route("/predict_elo", methods=["POST"])
def predict_elo():
    features = request.get_json()["features"]

    new_data = pd.DataFrame(features)
    predicted_elo = model.predict(new_data)
    
    predicted_elo_string = str(predicted_elo)
    
    return jsonify({"predicted_level": predicted_elo_string})


@app.route("/nnue_eval", methods=["POST"])
def nnue_evaluate():
    fen = request.get_json()["board"]
    engine = stockfish.Stockfish(stockfish_path)
    engine.set_fen_position(fen)
    evaluation = engine.get_evaluation()
    return jsonify({"evaluation": evaluation['value']})



@app.route("/nnue_policy", methods=["POST"])
def nnue_policy_network():
  """Returns a probability distribution over the set of legal moves."""
  fen = request.get_json()["fen"]
  # Create a Stockfish engine
  engine = stockfish.Stockfish(stockfish_path)

  # Set the FEN position in the Stockfish engine
  engine.set_fen_position(fen)

  # Get the best move from Stockfish
  best_move = engine.get_best_move()
  """
  # Get the list of legal moves
  board = chess.Board(fen)
  legal_moves = list(board.legal_moves)

  # Calculate the probability of each legal move
  policy_probabilities = {}
  for move in legal_moves:
    policy_probabilities[move] = 1.0 if move == best_move else 0.0

  # Normalize the policy probabilities
  sum_of_probabilities = sum(policy_probabilities.values())
  for move in legal_moves:
    policy_probabilities[move] /= sum_of_probabilities

  """  
  # Return the policy probabilities
  return jsonify({"best_move": best_move})



if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.23.0.2:5001/ (Press CTRL+C to quit)
192.168.1.7 - - [28/Oct/2023 23:36:58] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:39] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:40] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:40] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:41] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:42] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:42] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:43] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:43] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:44] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:45] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct/2023 23:37:45] "POST /nnue_policy HTTP/1.1" 200 -
192.168.1.7 - - [28/Oct

In [42]:
import stockfish
import chess

stockfish_path = "C:\\Users\\faisa\\Documents\\Games_Section\\chess\\AIs\\stockfish\\stockfish-windows-x86-64-avx2.exe"

def nnue_policy_network(fen):
  
  engine = stockfish.Stockfish(stockfish_path)

  engine.set_fen_position(fen)

  best_move = engine.get_best_move()

  board = chess.Board(fen)

  legal_moves = list(board.legal_moves)
  l_moves = [str(move) for move in legal_moves]


  policy_probabilities = {}
  for move in l_moves:
    policy_probabilities[move] = 1.0 if move == best_move else 0.0

  
  sum_of_probabilities = sum(policy_probabilities.values())
  for move in l_moves:
    policy_probabilities[move] /= sum_of_probabilities

  return policy_probabilities


In [44]:
x = nnue_policy_network("r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24")
print(x)

{'a8g8': 0.0, 'a8f8': 0.0, 'a8e8': 0.0, 'a8d8': 0.0, 'a8c8': 0.0, 'a8b8': 0.0, 'e7e8': 0.0, 'e7g7': 0.0, 'e7f7': 0.0, 'e7d7': 0.0, 'e7c7': 0.0, 'e7e6': 0.0, 'f2b6': 0.0, 'f2c5': 0.0, 'f2d4': 0.0, 'f2g3': 0.0, 'f2e3': 0.0, 'f2g1': 0.0, 'f2e1': 0.0, 'b2e5': 0.0, 'b2d4': 0.0, 'b2c3': 0.0, 'b2b3': 0.0, 'b2a3': 0.0, 'b2c2': 0.0, 'b2a2': 0.0, 'b2c1': 0.0, 'b2b1': 1.0, 'b2a1': 0.0, 'b7b6': 0.0, 'a7a6': 0.0, 'f6f5': 0.0, 'd5d4': 0.0, 'b7b5': 0.0, 'a7a5': 0.0}
